<a href="https://colab.research.google.com/github/ThanhHung2112/Indeed_crawler/blob/main/Skills_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy download en_core_web_lg

/opt/anaconda3/envs/rasa/bin/python: No module named spacy


In [3]:
import spacy
from spacy.training.example import Example
import json

In [5]:
tags = [
"Business_skill"
"ComputerScience_skill"
"DataScience_skill"
"Health_skill"
"PhysicalScienceAndengineering_skill"
"SocialSciences_skill"
"Artsandhumannities_skill"
"Informationtechnology_skill"
"languagelearning_skill"
"PersonalDevelopment_skill"
"Mathandlogic_skill"
]

In [28]:
import spacy
from spacy.training.example import Example
import json
import random
import warnings

# Suppress warnings (optional, use with caution)
warnings.filterwarnings("ignore", category=UserWarning, module="spacy")
warnings.filterwarnings("ignore", category=FutureWarning, module="tensorflow")

# Load training data from JSON file
def load_data_from_json(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        return data
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {file_path}: {e}")
        return None

# Load the spaCy English model
nlp = spacy.blank("en")

# Add the NER component to the pipeline
ner = nlp.add_pipe("ner")

# Add the labels (in this case, just "SKILL")
ner.add_label("skill")

# Disable other pipeline components for training efficiency
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# Specify the path to your JSON file
json_file_path = "300_minn.json"

# Load training data from JSON file
training_data = load_data_from_json(json_file_path)

if training_data:
    # Remove duplicate entities based on start and end positions
    cleaned_training_data = []
    seen_entities = set()

    for entry in training_data:
        labels_list = entry.get("label", [])

        # Ensure that labels is a list of dictionaries
        if isinstance(labels_list, str):
            labels_list = [{"text": labels_list, "start": -1, "end": -1, "labels": ["skill"]}]
        elif not isinstance(labels_list, list):
            continue  # Skip entries without a valid label list

        cleaned_labels_list = []
        for label in labels_list:
            start = label.get("start", -1)
            end = label.get("end", -1)
            label_text = label.get("labels", "")

            # Ensure unique start and end positions
            if (start, end) not in seen_entities:
                cleaned_labels_list.append({"text": label_text, "start": start, "end": end, "labels": ["skill"]})
                seen_entities.add((start, end))

        if cleaned_labels_list:
            cleaned_training_data.append({"text": entry.get("text", ""), "label": cleaned_labels_list})

    with nlp.disable_pipes(*other_pipes):
        # Initialize the training
        nlp.begin_training()

        # Iterate through the cleaned training data for multiple epochs
        for epoch in range(1000):
            # Shuffle the cleaned training data
            random.shuffle(cleaned_training_data)

            # Create batches of cleaned training data
            for entry in cleaned_training_data:
                text = entry.get("text", "")
                labels_list = entry.get("label", [])

                # Process annotations and extract entities
                entities = []
                for label in labels_list:
                    start = label.get("start", -1)
                    end = label.get("end", -1)
                    label_text = label.get("labels", "")
                    entities.append((start, end, label_text))

                # Create a spaCy Example
                example = Example.from_dict(nlp.make_doc(text), {"entities": entities})

                # Update the model with iterating each example
                nlp.update([example], drop=0.5)

    # Save the trained model to disk /content/drive/MyDrive/NER/
    nlp.to_disk("skills_ner_model")
else:
    print("Failed to load training data from the JSON file.")


In [29]:
# Load the saved model from disk
loaded_nlp = spacy.load("skills_ner_model")

# Test the loaded model with some example texts
test_texts = [
    "I am skilled in Python and Java programming.",
    "My experience includes using TensorFlow for machine learning.",
    "I have hands-on experience with MongoDB and MySQL.",
    "write full length feature film script course write complete feature length screenplay film television serious drama romantic comedy anything learn break creative process component discover structured process allow produce polished pitch ready script end course complete project increase confidence idea ability feel prepared pitch first script get start next course design tap creativity base active learn actual learning take place within activity write learn link trailer course view trailer please copy paste link browser https vimeocom b b dc learner review love approach professor wheeler take towards course point easy follow informative would definitely recommend anyone interested take screenplay writing course course curriculum simple adopt professional writer room process write post work peer review share feedback peer revise work feedback receive peer real world feel professional writer room yet prior experience writer require proponent experiential learn active learning lecture short sometimes two minute long point design step step process essential success script writer guide show write firmly believe way become writer write write write learner review would like thank course instructor amazing course need get start mention prior script writing experience require begin basic word processor week two choose download free scriptwriting software celtx trelby may choose purchase final draft industry standard continue use word processor script format learner review writer concern regard protection original work coursera privacy policy protect learner ip indeed sole owner work",
    "Build machine learning",
    "use java and sql to management database",
    """
    As Technical Account Manager you will support our internal and external clients in online marketing campaigns configuration. We are working on our own proprietary solution that is the main point of client’s Marketing Technology Stack, integrating data from various services and delivering this data to external providers.

Tasks

Analyzing client websites to make sure they have the data needed for online marketing campaigns.
Setting up, running, checking, and turning off various services for clients.
Solving issues with these services.
Working with our developer teams to handle reported problems, configure services, improve existing solutions, and brainstorm new ones.
Checking problem reports and passing them to the right teams with initial analysis results.
Looking at services from external vendors that we haven't used yet when we need them.
Completing tasks with clear documentation for reference in the future.
Opportunity to become the go-to person for a particular area of our knowledge base and manage internal team documentation.
Actively participating in team meetings where we all discuss team matters (Governance Meeting).
Requirements

You should know about internet browsers (understand what happens when you enter a website URL and hit "Enter").
You should be comfortable using Developer tools in browsers (able to analyze network traffic and inspect data layer - like JavaScript variables and DOM elements variables).
You should be familiar with Online Marketing tools, such as:
Principles of Tag management systems (like Google Tag Manager, which is a must, and others like Tealium or Commanders Act)
Analytics tools like Google Analytics
Online Marketing platforms, like Google Ads, Facebook Advertising, and others (Criteo, Google Marketing Platform, Microsoft Advertising, etc.)
Consent Management Platforms (CMP) like Usercentrics, Klaro, OneTrust, Didomi
You should have some knowledge of SQL (to write basic queries for database data retrieval).
You should have some understanding of JavaScript (or a willingness to learn) to handle monitoring code implementation (tracking data, processing, using variables from DOM or URL).
You should be knowledgeable about online privacy topics, especially regarding tracking codes and Privacy Enhanced Technologies in various browsers.
Fluency in English is a must.
You should be able to document your work so that you or anyone else can pick up where you left off and retain valuable information for future reference (knowledge capture).
Understanding processes related to IT service management (like ITIL) is a plus.
You should be a good team player, adhering to mutually agreed-upon rules and be able to communicate on technical topics with both technical and non-technical individuals.
Knowledge of the German language is an added advantage.
You'll help maintain the team and company's positive image.
You should understand and support the team's vision, to which you will be contributing.
Benefits

Work in a global team.
remote work first company.
Competitive pay.
Enjoy perks like a MultiSport card and private health coverage.
Grow professionally with learning opportunities.
Join a friendly, energetic team with a laid-back work environment
    """,
    "Client should have strong problem solving skills, and good at both reading and speaking English"

]

for text in test_texts:
    doc = loaded_nlp(text)
    print("Input Text:", text)
    print("Entities:", [(ent.text, ent.label_) for ent in doc.ents])
    print()


Input Text: I am skilled in Python and Java programming.
Entities: []

Input Text: My experience includes using TensorFlow for machine learning.
Entities: []

Input Text: I have hands-on experience with MongoDB and MySQL.
Entities: []

Input Text: write full length feature film script course write complete feature length screenplay film television serious drama romantic comedy anything learn break creative process component discover structured process allow produce polished pitch ready script end course complete project increase confidence idea ability feel prepared pitch first script get start next course design tap creativity base active learn actual learning take place within activity write learn link trailer course view trailer please copy paste link browser https vimeocom b b dc learner review love approach professor wheeler take towards course point easy follow informative would definitely recommend anyone interested take screenplay writing course course curriculum simple adopt p

In [30]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

/opt/anaconda3/envs/rasa/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/rasa/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if

[{'entity': 'B-PER', 'score': 0.9990139, 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': 0.999645, 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Ivo/emscad-skill-extraction")
model = AutoModelForSequenceClassification.from_pretrained("Ivo/emscad-skill-extraction")

In [31]:
# imports
import spacy
from spacy.matcher import PhraseMatcher

# load default skills data base
from skillNer.general_params import SKILL_DB
# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor

# init params of skill extractor
nlp = spacy.load("en_core_web_lg")
# init skill extractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

# extract skills from job_description
job_description = """
You are a Python developer with a solid experience in web development
and can manage projects. You quickly adapt to new environments
and speak fluently English and French
"""

annotations = skill_extractor.annotate(job_description)


loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [32]:
annotations

{'text': 'you are a python developer with a solid experience in web development and can manage projects you quickly adapt to new environments and speak fluently english and french',
 'results': {'full_matches': [{'skill_id': 'KS122Z36QK3N5097B5JH',
    'doc_node_value': 'web development',
    'score': 1,
    'doc_node_id': [10, 11]}],
  'ngram_scored': [{'skill_id': 'KS125LS6N7WP4S6SFTCK',
    'doc_node_id': [3],
    'doc_node_value': 'python',
    'type': 'fullUni',
    'score': 1,
    'len': 1},
   {'skill_id': 'KS1267F6MSPN366LX7ST',
    'doc_node_id': [14, 15],
    'doc_node_value': 'manage projects',
    'type': 'lowSurf',
    'score': 2,
    'len': 2},
   {'skill_id': 'KS120626HMWCXJWJC7VK',
    'doc_node_id': [18],
    'doc_node_value': 'adapt',
    'type': 'lowSurf',
    'score': 0.5245310068130493,
    'len': 1},
   {'skill_id': 'KS123K75YYK8VGH90NCS',
    'doc_node_id': [25],
    'doc_node_value': 'english',
    'type': 'lowSurf',
    'score': 1,
    'len': 1},
   {'skill_id'